In [1]:
import pandas as pd
from pandas import read_csv
import numpy as np
import random as rnd
import re

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
#from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier



from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
#from sklearn.metrics import ndcg_score
#from sklearn.metrics import label_ranking_average_precision_score
from matplotlib import pyplot

In [2]:
train = pd.read_csv('train_pre.csv', index_col = 0)
test = pd.read_csv('test_pre.csv', index_col = 0)
gender_df = pd.read_csv('gender_submission.csv', index_col = 0)
train.head()

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,0,3,1,1,0,0,0,23,0,2,0,1
1,1,1,0,2,0,3,1,51,1,2,0,3
2,1,3,0,1,0,1,0,22,0,1,1,2
3,1,1,0,2,0,3,0,44,1,2,0,3
4,0,3,1,2,0,1,0,24,0,1,1,1


In [3]:
train['Survived'].describe()

count    891.000000
mean       0.383838
std        0.486592
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

In [4]:
X = train.drop('Survived', 1)
y = train['Survived']
X_train, X_test, y_train, y_test =  train_test_split(
    X, y, test_size=0.3, random_state=0)
X.head()

,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,3,1,1,0,0,0,23,0,2,0,1
1,1,0,2,0,3,1,51,1,2,0,3
2,3,0,1,0,1,0,22,0,1,1,2
3,1,0,2,0,3,0,44,1,2,0,3
4,3,1,2,0,1,0,24,0,1,1,1


In [5]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X_train, y_train, cv=5
#                         , scoring='f1_macro'  # be default score is accuracy
                        )
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.82 accuracy with a standard deviation of 0.02


In [6]:
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing

clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))
scores = cross_val_score(clf, X, y, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.83 accuracy with a standard deviation of 0.02


#### cross_validate function differs from cross_val_score in two ways <br>
* It allows specifying multiple metrics for evaluation. <br>
* It returns a dict containing training scores, fit-times and score-times in addition to the test score. <br>


In [7]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

scores = cross_validate(clf, X, y,
                       scoring='accuracy', cv=5,
                       return_estimator=True, return_train_score=True)
sorted(scores.keys())
scores['train_score']

array([0.84831461, 0.8569425 , 0.84291725, 0.8513324 , 0.83730715])

In [8]:
scores['test_score']
#print("%0.2f accuracy with a standard deviation of %0.2f" % (scores['test_score'].mean(), scores['test_score'].std()))

array([0.84357542, 0.80898876, 0.8258427 , 0.80337079, 0.85955056])

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        svm.SVC(), param_grid, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.811 (+/-0.049) for {'C': 1, 'kernel': 'linear'}
0.813 (+/-0.057) for {'C': 10, 'kernel': 'linear'}
0.811 (+/-0.049) for {'C': 100, 'kernel': 'linear'}
0.811 (+/-0.049) for {'C': 1000, 'kernel': 'linear'}
0.713 (+/-0.102) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.714 (+/-0.113) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.818 (+/-0.055) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.732 (+/-0.083) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.821 (+/-0.060) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.815 (+/-0.061) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.807 (+/-0.053) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.814 (+/-0.060) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed 

In [9]:
from pprint import pprint
from time import time
import logging
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# pipeline defines estimator
pipe = Pipeline([
    ('rf',RandomForestClassifier(random_state = 42)),
    ('clf', SGDClassifier()),
])

parameters = {
    'rf__n_estimators':[120, 140],
    'rf__max_depth': [30, 50],
    'rf__min_samples_split': [2,3], 
    'rf__min_samples_leaf': [3, 5],  
    'rf__class_weight': [{0:1, 1:1}, {0:1, 1:5}, {0:1, 1:3}, 'balanced'], 
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__max_iter': (10, 50, 80),
}

if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipe.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    y_true, y_pred = y_test, grid_search.predict(X_test)
    print(classification_report(y_true, y_pred))

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'RandomForestClassifier(random_state=42)' (type <class 'sklearn.ensemble._forest.RandomForestClassifier'>) doesn't

In [ ]:
# Grid Search for Random Forest
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}
# Create a based model
rf = RandomForestClassifier()
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(X_train, y_train)
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
print("best parameter :", best_parameters)
        
y_true, y_pred = y_test, grid_search.predict(X_test)
print(classification_report(y_true, y_pred))

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
# Create a based model
clf = RandomForestClassifier()

# Number of trees in Random Forest
rf_n_estimators = [int(x) for x in np.linspace(200, 1000, 5)]
rf_n_estimators.append(1500)
rf_n_estimators.append(2000)

# Maximum number of levels in tree
rf_max_depth = [int(x) for x in np.linspace(5, 55, 11)]
# Add the default as a possible value
rf_max_depth.append(None)

# Number of features to consider at every split
rf_max_features = ['auto', 'sqrt', 'log2']

# Criterion to split on
rf_criterion = ['mse', 'mae']

# Minimum number of samples required to split a node
rf_min_samples_split = [int(x) for x in np.linspace(2, 10, 9)]

# Minimum decrease in impurity required for split to happen
rf_min_impurity_decrease = [0.0, 0.05, 0.1]

# Method of selecting samples for training each tree
rf_bootstrap = [True, False]

# Create the grid
rf_dist = {'n_estimators': rf_n_estimators,
               'max_depth': rf_max_depth,
               'max_features': rf_max_features,
               'criterion': rf_criterion,
               'min_samples_split': rf_min_samples_split,
               'min_impurity_decrease': rf_min_impurity_decrease,
               'bootstrap': rf_bootstrap}


n_iter_search = 20
random_search= RandomizedSearchCV(estimator = clf, param_distributions = random_grid, random_state = 42
                                ,n_iter=n_iter_search, cv = 3, verbose = 2, n_jobs = -1)
start = time()
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
print(random_search.best_params_)

In [16]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
from time import time

grid_n_estimator = [10, 50, 100, 300]
grid_ratio = [.1, .25, .5, .75, 1.0]
grid_learn = [.01, .03, .05, .1, .25]
grid_max_depth = [2, 4, 6, 8, 10, None]
grid_min_samples = [5, 10, .03, .05, .10]
grid_criterion = ['gini', 'entropy']
grid_bool = [True, False]
grid_seed = [0]

model_list = [
    ('rfc', RandomForestClassifier())
    ,('xgb', XGBClassifier() )
]

param_lib = [
 
            #http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier
          [{'n_estimators': grid_n_estimator, #default=10
            'criterion': grid_criterion, #default=”gini”
            'max_depth': grid_max_depth, #default=None
            'oob_score': [True], #default=False -- 12/31/17 set to reduce runtime -- The best parameter for RandomForestClassifier is {'criterion': 'entropy', 'max_depth': 6, 'n_estimators': 100, 'oob_score': True, 'random_state': 0} with a runtime of 146.35 seconds.
            'random_state': grid_seed 
           }],
    
            #http://xgboost.readthedocs.io/en/latest/parameter.html
          [{
            #XGBClassifier - http://xgboost.readthedocs.io/en/latest/parameter.html
            'learning_rate': grid_learn, #default: .3
            'max_depth': [1,2,4,6,8,10], #default 2
            'n_estimators': grid_n_estimator, 
            'seed': grid_seed  
             }]   

        ]

n_iter_search = 20
for model, param in zip(model_list, param_lib):
    grid_search= GridSearchCV(estimator = model[1], param_grid = param
                               ,cv = 5, verbose = 2, n_jobs = -1)
    start = time()
    grid_search.fit(X_train, y_train)
    best_parameters = grid_search.best_estimator_.get_params()
    print('Best parameters: ')
#    for param_name in sorted(param.keys()):
#        print("\t%s: %r" % (param_name, best_parameters[param_name]))
    print(best_parameters)    
    y_true, y_pred = y_test, grid_search.predict(X_test)
    print(classification_report(y_true, y_pred))


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters: 
{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 4, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 300, 'n_jobs': None, 'oob_score': True, 'random_state': 0, 'verbose': 0, 'warm_start': False}
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       168
           1       0.77      0.75      0.76       100

    accuracy                           0.82       268
   macro avg       0.81      0.81      0.81       268
weighted avg       0.82      0.82      0.82       268

Fitting 5 folds for each of 120 candidates, totalling 600 fits
[16:54:29] WARNING: /opt/concourse/worker/volumes/live/7a2b9f41-3287-451b-6691-43e9a6c0910f/volume/xgboost-split_

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier

class HyperModel : 
    def __init(self, model_type = 'classification', model_name = 'random_forest'
               , model_tuning = 'None', **kwargs):
        self.model = None
        self.model_type = model_type # 'classification', 'regression'
        self.model_name = model_name
        self.model_tuning = model_tuning
        self.set_model(model_type, model_name, kwargs)
    
    def _set_model(self, model_type, model_name, kwargs):
        if model_type == 'classification':
            if model_name == 'random_forest':
                self.model = RandomForestClassifier()
        elif model_type == 'regression':
            if model_name == 'random_forest':
                self.model = RandomForestRegressor(**{key:val for key, val in kwargs.items()
                                                  if key in ['n_estimators', 'max_depth',
                                                             'max_features', 'min_samples_leaf',
                                                             'min_samples_split',
                                                             'n_jobs']})
        else:
            raise ValueError(f'{model_type} not supported')   
    def _tune_model(self, model_type, model_name, model_tuning, kwargs):
        if model

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier


class HyperModel :
    def __init__(self, model_type = 'classification', model_name = 'random_forest', **kwargs):
        self.model = None
        self.model_type = model_type # 'classification', 'regression'
        self.model_name = model_name
        self.set_model(model_type, kwargs)
        
    def _set_model(self, model_type, kwargs):
        if model_type == 'classification':
            if model_type == 'xgboost':
            self.model = XGBClassifier(**{key:val for key, val in kwargs.items()
                                         if key in ['n_estimators', 'max_depth',
                                                    'learning_rate', 'gamma',
                                                    'n_jobs']})
            elif model_type == 'random_forest':
                self.model = RandomForestClassifier(**{key:val for key, val in kwargs.items()
                                                      if key in ['n_estimators', 'max_depth',
                                                                 'max_features', 'min_samples_leaf',
                                                                 'min_samples_split',
                                                                 'n_jobs']})
            else:
                raise ValueError(f'{model_name} not supported')

        elif model_type == 'regression':
            if model_name == 'random_forest':
                self.model = RandomForestRegressor(**{key:val for key, val in kwargs.items()
                                                  if key in ['n_estimators', 'max_depth',
                                                             'max_features', 'min_samples_leaf',
                                                             'min_samples_split',
                                                             'n_jobs']})
        else:
            raise ValueError(f'{model_type} not supported')
            
    def fit(self, X, y, **kwargs):
        return self.model.fit(X, y, **kwargs)

    def predict(self, X):
        return self.model.predict(X)

    def get_params(self, deep=False):
        params =  self.model.get_params(deep)
        params['model_type'] = self.model_type
        return params
    
     def set_params(self, **params):
        if params:
            self.model_type = params['model_type']
            del params['model_type']
            self._set_model(self.model_type, params)

        return self.model

In [17]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer

#from hyper_model import HyperModel